In [1]:
import pandas as pd
import numpy as np
import torch  # Add this import
import torch.nn as nn
import torch.optim as optim
# Move model and data to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

aicup_train = pd.read_csv(r"C:\Users\et\Desktop\新增資料夾\aicup\analysised.csv")
aicup_train.drop(columns=['Unnamed: 0',"DateTime"], inplace=True)
aicup_train_features = aicup_train.drop(columns=["Power(mW)"])
aicup_train_target = aicup_train["Power(mW)"]
# Define the model
class PredictModel(nn.Module):
    def __init__(self):
        super(PredictModel, self).__init__()
        self.fc1 = nn.Linear(aicup_train_features.shape[1], 1024)
        self.fc2 = nn.Linear(1024,1024)
        self.fc3 = nn.Linear(1024, 1024)
        self.fc4 = nn.Linear(1024, 1024)
        self.fc5 = nn.Linear(1024, 1024)
        self.fc6 = nn.Linear(1024, 64)
        self.fc7 = nn.Linear(64, 64)
        self.fc8 = nn.Linear(64, 64)
        self.fc9 = nn.Linear(64, 64)
        self.fc10 = nn.Linear(64, 64)
        self.fc11= nn.Linear(64, 64)
        self.fc12 = nn.Linear(64, 64)
        self.fc13 = nn.Linear(64, 64)
        self.fc14 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

predict_model = PredictModel().cuda()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(predict_model.parameters(), lr=0.001)



Using device: cuda


In [2]:
torch.__version__

'2.4.1+cu121'

In [3]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(aicup_train_features, aicup_train_target, test_size=0.2, random_state=42)

print(f"Training features shape: {X_train.shape}")
print(f"Testing features shape: {X_test.shape}")
print(f"Training target shape: {y_train.shape}")
print(f"Testing target shape: {y_test.shape}")

Training features shape: (847600, 4)
Testing features shape: (211901, 4)
Training target shape: (847600,)
Testing target shape: (211901,)


In [4]:
# Convert the training data to PyTorch tensors and move to the same device as the model
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)

# Training loop

num_epochs = 1000

for epoch in range(num_epochs):
    # Forward pass
    outputs = predict_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

c:\Users\et\Desktop\新增資料夾\.conda\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([847600, 1])) that is different to the input size (torch.Size([847600, 1024])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/1000], Loss: 2675892.0000
Epoch [2/1000], Loss: 1047186.4375
Epoch [3/1000], Loss: 616521.8125
Epoch [4/1000], Loss: 878062.0625
Epoch [5/1000], Loss: 1045450.2500
Epoch [6/1000], Loss: 799145.6875
Epoch [7/1000], Loss: 470004.8438
Epoch [8/1000], Loss: 339103.9688
Epoch [9/1000], Loss: 393101.6562
Epoch [10/1000], Loss: 435345.4375
Epoch [11/1000], Loss: 374668.7500
Epoch [12/1000], Loss: 278855.2188
Epoch [13/1000], Loss: 234362.0938
Epoch [14/1000], Loss: 243571.1406
Epoch [15/1000], Loss: 247316.0469
Epoch [16/1000], Loss: 212949.7812
Epoch [17/1000], Loss: 164021.9219
Epoch [18/1000], Loss: 136894.1406
Epoch [19/1000], Loss: 135699.5156
Epoch [20/1000], Loss: 137402.4688
Epoch [21/1000], Loss: 126032.8047
Epoch [22/1000], Loss: 106970.7031
Epoch [23/1000], Loss: 92472.2891
Epoch [24/1000], Loss: 84762.5156
Epoch [25/1000], Loss: 77970.6641
Epoch [26/1000], Loss: 69653.9688
Epoch [27/1000], Loss: 62802.2852
Epoch [28/1000], Loss: 58792.3047
Epoch [29/1000], Loss: 54573.019

KeyboardInterrupt: 

In [5]:
import torch

# Convert the testing data to PyTorch tensors and move to the same device as the model
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

# Set the model to evaluation mode
predict_model.eval()

# Forward pass on the test set
with torch.no_grad():
    test_outputs = predict_model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)

print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 808.8356


c:\Users\et\Desktop\新增資料夾\.conda\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([211901, 1])) that is different to the input size (torch.Size([211901, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
